In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very low-precision encoding (`precision=2`) and a very small field (`order=127`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

Player 1: --> 2 PRZS 9180253633517821387
Player 2: --> 0 PRZS 2570339128807744623
Player 0: --> 1 PRZS 3167425176742296127
Player 2: <-- 1 PRZS 9180253633517821387
Player 0: <-- 2 PRZS 2570339128807744623
Player 1: <-- 0 PRZS 3167425176742296127
Player 2: --> 0 GATHER 47
Player 1: --> 0 GATHER 75
Player 0: --> 0 GATHER 25
Player 2: --> 1 GATHER 47
Player 1: --> 1 GATHER 75
Player 0: <-- 2 GATHER 47
Player 2: --> 2 GATHER 47
Player 1: <-- 2 GATHER 47
Player 0: <-- 0 GATHER 25
Player 2: <-- 2 GATHER 47
Player 1: <-- 1 GATHER 75
Player 0: <-- 1 GATHER 75
Player 0: --> 1 GATHER 25
Player 0: --> 2 GATHER 25
Player 1: <-- 0 GATHER 25
Player 2: <-- 0 GATHER 25
Player 1: --> 2 GATHER 75
Player 2: <-- 1 GATHER 75


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    handler = transcript.net_handler()
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 0: --> 1 PRZS 2725862109994643561
Player 1: --> 2 PRZS 4394038262397323354
Player 2: --> 0 PRZS 5465098583386923265
Player 1: <-- 0 PRZS 2725862109994643561
Player 0: <-- 2 PRZS 5465098583386923265
Player 2: <-- 1 PRZS 4394038262397323354
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 1: --> 0 GATHER 48
Player 0: Let's reveal the results!
Player 2: --> 0 GATHER 88
Player 0: --> 0 GATHER 11
Player 1: --> 1 GATHER 48
Player 2: --> 1 GATHER 88
Player 0: <-- 1 GATHER 48
Player 1: <-- 1 GATHER 48
Player 2: --> 2 GATHER 88
Player 1: <-- 2 GATHER 88
Player 0: <-- 2 GATHER 88
Player 2: <-- 2 GATHER 88
Player 0: <-- 0 GATHER 11
Play

In [4]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 0,sent,0,1,PRZS,4771570285743758687
Player 1,sent,1,2,PRZS,5045449825072802068
Player 2,sent,2,0,PRZS,1552344417222398375
Player 1,received,0,1,PRZS,4771570285743758687
Player 0,received,2,0,PRZS,1552344417222398375
Player 2,received,1,2,PRZS,5045449825072802068
# Player 1: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 1: Let's add some secrets!
# Player 2: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 1: Let's reveal the results!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
Player 1,sent,1,0,GATHER,48
Player 2,sent,2,0,GATHER,98
Player 0,sent,0,0,GATHER,1
Player 1,sent,1,1,GATHER,48
Player 0,received,1,0,GATHER,48
Player 2,sent,2,1,GATHER,98
Player 1,received,1,1,GATHER,48
Player 0,received,0,0,GATHER,1
Player 2,sent,2,2,GATHER,98
Player

If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    fmt = "# {processName}: {msg}"
    netfmt = "{processName},{net.verb},{net.src},{net.dst},{net.tag},{net.payload}"
    handler = transcript.net_handler(received=False, fmt=fmt, netfmt=netfmt)
    transcript.set_handler(logging.getLogger(), handler)
    SocketCommunicator.run(fn=main, world_size=3);

# Player 0: Let's setup additive sharing!
# Player 1: Let's setup additive sharing!
# Player 2: Let's setup additive sharing!
Player 1,sent,1,2,PRZS,6249644603896891259
Player 2,sent,2,0,PRZS,7119465610283473314
Player 0,sent,0,1,PRZS,2526812129119959745
# Player 1: Let's share some secrets!
# Player 2: Let's share some secrets!
# Player 0: Let's share some secrets!
# Player 2: Let's add some secrets!
# Player 1: Let's add some secrets!
# Player 0: Let's add some secrets!
# Player 2: Let's reveal the results!
# Player 0: Let's reveal the results!
# Player 1: Let's reveal the results!
Player 2,sent,2,0,GATHER,101
Player 0,sent,0,0,GATHER,89
Player 1,sent,1,0,GATHER,84
Player 2,sent,2,1,GATHER,101
Player 1,sent,1,1,GATHER,84
Player 2,sent,2,2,GATHER,101
Player 0,sent,0,1,GATHER,89
Player 0,sent,0,2,GATHER,89
Player 1,sent,1,2,GATHER,84


In [6]:
from cicada.arithmetic import Field

with transcript.record():
    handler = transcript.code_handler()
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 154406471902159217769667420935286109852, 'inc': 335460593912382519142344441987828168893}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([106, 17, 119], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([106, 17, 119], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([107, 18, 120], dtype='object'))



In [7]:
import io

with transcript.record():
    buffer = io.StringIO()
    handler = transcript.code_handler(logging.StreamHandler(buffer))
    transcript.set_handler(logging.getLogger(), handler)
    
    f = Field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    f.inplace_add(a, b)

In [8]:
buffer.seek(0)
for line in buffer:
    print(line.strip())

cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 46523294019198452464395772408159304117, 'inc': 64391502372486286024740319327295746511}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([124, 56, 85], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
cicada.arithmetic.Field(order=127).inplace_add(lhs=lhs, rhs=numpy.array([124, 56, 85], dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array([125, 57, 86], dtype='object'))



Now, we can execute the buffered transcript, one-line-at-a-time, and confirm that none of the assertions raise exceptions:

In [9]:
import cicada.transcript

try:
    buffer.seek(0)
    for line in buffer:
        exec(line)
except Exception as e:
    print(f"Consistency verification failed: {e}")
else:
    print("Consistency verification succeeded.")

Consistency verification succeeded.


Let's go back to our original example. Because it requires three players, we'll setup Cicada to write a separate transcript for each player:  

In [10]:
def main(comm):
    handler = transcript.code_handler(logging.FileHandler(f"player-{comm.rank}.py", "w"))
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Note that we're configuring the transcript logging inside `main`, since that allows us to generate a filename based on the player rank. Also note that we've left our context messages in-place. Let's see what the results look like for one player:

In [11]:
with open("player-0.py") as stream:
    for line in stream:
        print(line.strip())

# Let's setup additive sharing!
# Let's share some secrets!
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 334906897946999562783477543067812248234, 'inc': 268301515194818259492276875971024363035}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(62, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 224012827719984138909395568455464289619, 'inc': 32988247567743730537229980144464350827}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(47, dtype='object'))

lhs = numpy.array(62, dtype='object')
cicada.arithmetic.Field(order=127).inplace_subtract(lhs=lhs, rhs=numpy.array(47, dtype='object'))
cicada.transcript.assert_equal(lhs, numpy.array(15, dtype='object'))

cicada.transc

And now let's run all three transcripts to verify their consistency:

In [12]:
import traceback

for rank in range(3):
    try:
        with open(f"player-{rank}.py") as stream:
            for line in stream:
                exec(line)
    except Exception as e:
        print(f"Player {rank} consistency verification failed: {e}")
        print(traceback.format_exc())
    else:
        print(f"Player {rank} consistency verification succeeded.")

Player 0 consistency verification succeeded.
Player 1 consistency verification succeeded.
Player 2 consistency verification succeeded.


In [13]:
def main(comm):
    handler = logging.StreamHandler() if comm.rank == 0 else logging.NullHandler()
    handler = transcript.code_handler(handler, sent=True, received=True)
    transcript.set_handler(logging.getLogger(), handler)

    transcript.log("Let's setup additive sharing!")
    encoding = FixedPoint(precision=2)
    protocol = AdditiveProtocolSuite(comm, order=127, encoding=encoding)
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

# Let's setup additive sharing!
# 0 --> 1 PRZS 8149388949109382716
# 0 <-- 2 PRZS 7371382083269188734
# 0 <-- 2 GATHER 11
# Let's share some secrets!
# 0 <-- 1 GATHER 81
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 203431636371358734958945601876225668761, 'inc': 244270650531735514959971537900406170281}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(58, dtype='object'))

bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 29948618504954213903035899642141407178, 'inc': 336424436022423810171075766137774000049}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.Field(order=127).uniform(size=(), generator=numpy.random.Generator(bg)), numpy.array(111, dtype='object'))

lhs = numpy.array(58, dtype='object')
cicada.arithmetic.Field(order=127).inplace_subtract(lhs=lhs, rhs=numpy